In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
project = 'sompo'
label_text_path = './datasets/label_text_4855.json'
use_wavenet = True

if use_wavenet:
    weights_path = './logdir/dc3/train/keras/basemodel_retrain_215000.h5'
else:
    weights_path = './logdir/15_mar/sompo/keras/basemodel_retrain_195000.h5'
    config_path = './logdir/15_mar/sompo/keras/basemodel.json'



if project == 'sompo':
    img_path_regex = './datasets/sompo/April_1/sompo_test_1_april/imgs/*'

    ann_path_regex = './datasets/sompo/April_1/sompo_test_1_april/ann/*.json'
    
    output_dir_report = './report/sompo/'

elif project == 'dc3':
    
    img_path_regex = './datasets/dc3/test/imgs/*'

    ann_path_regex = './datasets/dc3/test/ann/*.json'

    output_dir_report = './report/dc3/'


output_dir_image = output_dir_report+'/eval_image'

batch_size = 32

# Get data

# load samples


In [3]:
from data_utils import create_tf_dataset
from data_utils import run_data_init
from glob import glob
from PIL import Image
import pandas
import os
import matplotlib.pyplot as plt
import cv2
import json
import pandas
from utils.ctc_bestpath import ctcBestPath
import numpy as np
from nltk.metrics.distance import edit_distance
from tqdm import tqdm
import xlsxwriter
import time
import tensorflow as tf
from pyson.utils import show_df
from text_normalizer.utils.normalize import normalize_text
from data_utils import read_image as load_image
from fuzzywuzzy.fuzz import ratio
import tensorflow as tf




def get_shape(box):
    """
        Arguments:
            box: QA label dictionary of {'shape_attributes':something, 'region_attributes':something}
            origin_image: full page image document
        return:
            ndarray: small image contain the text 
    """
    shape_attribute = box['shape_attributes']
    shape_type = shape_attribute['name']
    if shape_type == 'polygon' or shape_type=='polyline':
        all_points_x = shape_attribute['all_points_x']
        all_points_y = shape_attribute['all_points_y']
        x = np.min(all_points_x)
        max_x = np.max(all_points_x)
        y = np.min(all_points_y)
        max_y = np.max(all_points_y)
        w = max_x - x
        h = max_y - y
    elif shape_type == 'rect':
        x, y = shape_attribute['x'], shape_attribute['y']
        w, h = shape_attribute['width'], shape_attribute['height']
        
    else:
        raise Exception('shape_type : {} not yet process'.format(shape_type))
    return (x,y,w,h)



def read_qa_json(json_path):
    d = json.load(open(json_path, 'r', encoding='utf-8'))
    d = list(d['_via_img_metadata'].values())
    if len(d) > 0:
        return d[0]['regions']
    else:
        return None



def get_img_path(name, img_paths):
    for img_path in img_paths:
        img_name = os.path.basename(img_path).split('.')[0]
        if img_name == name:
            return img_path
    return None

# def caculate_ratio(row):
#     try:
#         norm_pred = _normalize_text(row['pred'])
#         norm_text = _normalize_text(row['text'])
#         return ratio(norm_pred, norm_text )
#     except:
#         return 0
    
K = tf.keras.backend
sess = K.get_session()

def write_to_excel(df, split=True):
    def export(file_name, df_file_name):
        summary = {}
        summary[file_name] = {'by_char': np.mean(df_file_name['ratio']), 'by_field': np.mean(df_file_name['byfield'])}
        
        workbook = xlsxwriter.Workbook(os.path.join(output_dir_report,'ocr_report_{}.xlsx'.format(file_name)))
        worksheet = workbook.add_worksheet()
        worksheet.write('A1', 'Image')
        worksheet.write('B1', 'Predict')
        worksheet.write('C1', 'Label')
        worksheet.write('D1', 'By Char')
        worksheet.write('E1', 'By Filed')
        
        for i, index in enumerate(df_file_name.index):
            i = i *2 
            row = df_file_name.loc[index]
            worksheet.insert_image('A{}'.format(i+2), row['path'], {'x_scale': 0.4, 'y_scale': 0.4})
            
            worksheet.write('B{}'.format(i + 2), str(row['pred']))

            worksheet.write('C{}'.format(i + 2), str(row['text']))

            worksheet.write('D{}'.format(i + 2), str(row['ratio']))
            
            worksheet.write('E{}'.format(i + 2), str(float(row['byfield'])))



        workbook.close()
        return summary
    
    summary = {}
    if split:
        file_names = set(df['filename'].values)

        for file_name in file_names:
            df_file_name = df[df['filename'] == file_name]
            summary.update(export(file_name, df_file_name))
            
        summary['average-char'] = df['ratio'].mean()
        summary['average-field'] = df['by_field'].mean()
        pandas.DataFrame(summary).transpose().sort_values(by=['by_char']).to_excel(os.path.join(output_dir_report, 'summary.xlsx'))
    else:
        summary.update(export('fuse_report', df))



In [4]:
K.set_learning_phase(0)

In [ ]:
from wavenet import build_model

old_config_path = './logdir/15_mar/sompo/keras/basemodel.json'
label_text = json.load(open('./datasets/label_text_4855.json', 'r'))
label_text = {int(l):t for l, t in label_text.items()}

In [ ]:
img_paths = glob(img_path_regex)
ann_paths = glob(ann_path_regex)
print(len(img_paths), len(ann_paths))

97 96


# Export detailed 

In [ ]:

output_small_df_path = os.path.join(output_dir_report, 'label.json')
if not os.path.exists(output_small_df_path):
    os.makedirs(output_dir_image, exist_ok=True)
    data = {}
    process_bar = tqdm(ann_paths)
    
    for ann_path in process_bar:
        img_name = os.path.basename(ann_path).split('.')[0]
        img_path = get_img_path(img_name, img_paths)
        if img_path:
            img = cv2.imread(img_path)
            lst_boxes = read_qa_json(ann_path)
            for i, box in enumerate(lst_boxes):
                (x,y,w,h) = get_shape(box)
                small_image = img[y:y+h, x:x+w]
                label = box['region_attributes']['label']
                small_img_name = '{}-{}.png'.format(img_name, i)
                small_img_path = os.path.join(output_dir_image, small_img_name)
                try:
                    assert cv2.imwrite(small_img_path, small_image)
                    data[small_img_path] = {'text': label, 'width': w, 'height':h, 'path': small_img_path, 'filename': img_name}
                except Exception as e:
                    print(e)
                    print(small_image.shape)


    df = pandas.DataFrame.from_dict(data).transpose()
    
    df.to_json(output_small_df_path)
else:
    df = pandas.read_json(output_small_df_path)

In [ ]:
batch_size = 128
df = df.sort_values(by=['width'])

In [ ]:
from data_utils import convert_batch_int_to_text

In [ ]:
if use_wavenet:
    from build_wavenet import basemodel
else:
    config = json.load(open(config_path))
    basemodel = tf.keras.models.Model.from_config(config)
    basemodel.load_weights(weights_path)
    
input_tensors, initer = create_tf_dataset(df, batch_size, mode='test')
preds_tensor = basemodel(input_tensors[0])

BUILD KUSH CNN


In [ ]:
data = df.transpose().to_dict()
run_data_init([initer], False, sess, batch_size)
preds = basemodel(input_tensors[0])

progress_bar = tqdm(range(len(df)//batch_size))
progress_bar.update(2)
for i in progress_bar:
    pred, bdata = sess.run([preds_tensor, {'inputs': input_tensors}])
    
    bpreds = [ctcBestPath(_, label_text) for _ in pred]
    bidxs = [_.decode('utf-8') for _ in bdata['inputs'][1]]
    for idx, pred in zip(bidxs, bpreds):
        data[idx]['pred'] = pred
    progress_bar.set_description('width: {}'.format(bdata['inputs'][0].shape[-2]))
    

In [ ]:
df = pandas.DataFrame.from_dict(data).transpose()

In [ ]:
import swifter

In [ ]:
def _normalize_text(text):
    convert_dict = {
        'd1':'dl',
        'U/1':'U/l',
        ' ': '',
        '十':'+'
    }
    text=  str(text)
    for k, v in convert_dict.items():
        text = text.replace(k, v)
    return normalize_text(text)

In [ ]:
df['pred_norm'] = df['pred'].swifter.apply(_normalize_text)
df['text_norm'] = df['text'].swifter.apply(_normalize_text)

In [ ]:
def rate(row):
    try:
        return ratio(row['pred_norm'], row['text_norm'])
    except:
        return 0

In [ ]:
df = df[df['text'].map(lambda x: not '??' in x)]

In [27]:
df['ratio'] = df[['pred_norm', 'text_norm']].swifter.apply(rate, axis=1)
df['by_field'] = df['ratio']==100

Pandas Apply: 100%|██████████| 25657/25657 [00:00<00:00, 35621.77it/s]


In [28]:
print('By char: {}\t By filed: {}'.format(df['ratio'].mean(), df['by_field'].mean()))

By char: 95.53490275558327	 By filed: 0.8785126865962505


In [ ]:
df.to_json(output_small_df_path)

In [ ]:
df_show = df[df['ratio']<100]
df_show = df_show[['pred_norm', 'text_norm', 'ratio','path']]
print(len(df_show))
show_df(df_show.sample(100), 'path')

In [ ]:
output_dir_report = output_dir_report[:-1]+output_dir_report[-1].replace('/', '')
os.system('zip -r {}.zip {}/*.xlsx'.format(output_dir_report, output_dir_report))

# Export excel files

In [29]:
write_to_excel(df)

In [ ]:
write_to_excel(df[df['ratio']==0], split=False)

In [31]:
import shutil

In [35]:
shutil.copy(weights_path, os.path.join(output_dir_report, os.path.basename(weights_path)))
shutil.copy(old_config_path, os.path.join(output_dir_report, os.path.basename(output_dir_report)))
shutil.copy(label_text_path, os.path.join(output_dir_report, os.path.basename(label_text_path)))

'./report/sompo/label_text_4855.json'

In [37]:
!ls report/sompo

basemodel_retrain_215000.h5  ocr_report_16-3_1.xlsx  ocr_report_24-3_1.xlsx
eval_image		     ocr_report_16-3_2.xlsx  ocr_report_27-1_0.xlsx
label.json		     ocr_report_16-3_3.xlsx  ocr_report_27-1_1.xlsx
label_text_4855.json	     ocr_report_17-1_0.xlsx  ocr_report_27-2_0.xlsx
ocr_report_14-1_0.xlsx	     ocr_report_17-1_1.xlsx  ocr_report_27-2_1.xlsx
ocr_report_14-1_1.xlsx	     ocr_report_17-2_0.xlsx  ocr_report_27-3_0.xlsx
ocr_report_14-2_0.xlsx	     ocr_report_17-2_1.xlsx  ocr_report_27-3_1.xlsx
ocr_report_14-2_1.xlsx	     ocr_report_17-2_2.xlsx  ocr_report_29-1_0.xlsx
ocr_report_14-3_0.xlsx	     ocr_report_17-2_3.xlsx  ocr_report_29-1_1.xlsx
ocr_report_14-3_1.xlsx	     ocr_report_17-3_0.xlsx  ocr_report_29-2_0.xlsx
ocr_report_15-1_0.xlsx	     ocr_report_17-3_1.xlsx  ocr_report_29-2_1.xlsx
ocr_report_15-1_1.xlsx	     ocr_report_18-1_0.xlsx  ocr_report_29-3_0.xlsx
ocr_report_15-1_2.xlsx	     ocr_report_18-1_1.xlsx  ocr_report_29-3_1.xlsx
ocr_report_15-1_3.xlsx	     ocr_report_18-2_0.xl

In [39]:
!apt-get install zip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libwrap0 libxmuu1 ncurses-term python3-chardet python3-pkg-resources
  python3-requests python3-six python3-urllib3 tcpd wget xauth
Use 'apt autoremove' to remove them.
The following NEW packages will be installed:
  zip
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 158 kB of archives.
After this operation, 587 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu xenial/main amd64 zip amd64 3.0-11 [158 kB]
Fetched 158 kB in 0s (199 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
Selecting previously unselected package zip.
(Reading database .

In [40]:
!zip -r report/sompo.zip report/sompo

  adding: report/sompo/ (stored 0%)
  adding: report/sompo/ocr_report_15-1_1.xlsx (deflated 1%)
  adding: report/sompo/ocr_report_14-1_0.xlsx (deflated 5%)
  adding: report/sompo/ocr_report_3-3_6.xlsx (deflated 2%)
  adding: report/sompo/ocr_report_17-2_2.xlsx (deflated 1%)
  adding: report/sompo/ocr_report_16-1_2.xlsx (deflated 2%)
  adding: report/sompo/ocr_report_15-1_6.xlsx (deflated 1%)
  adding: report/sompo/ocr_report_24-2_1.xlsx (deflated 4%)
  adding: report/sompo/ocr_report_3-3_4.xlsx (deflated 3%)
  adding: report/sompo/ocr_report_16-3_3.xlsx (deflated 2%)
  adding: report/sompo/ocr_report_15-1_3.xlsx (deflated 5%)
  adding: report/sompo/ocr_report_15-2_7.xlsx (deflated 1%)
  adding: report/sompo/ocr_report_3-2_6.xlsx (deflated 2%)
  adding: report/sompo/ocr_report_14-1_1.xlsx (deflated 1%)
  adding: report/sompo/ocr_report_15-1_5.xlsx (deflated 1%)
  adding: report/sompo/ocr_report_3-3_5.xlsx (deflated 3%)
  adding: report/sompo/ocr_report_14-2_0.xlsx (deflated 4%)
  adding

# Export summary master file

Download box.zip and upload onto: https://cinnamon-lab.app.box.com/folder/64756421862

In [ ]:
# df = df.sort_values(by=['ratio'])